In [1]:
# State クラス
class State:
    def __init__(self,
                 sender,
                 program,
                 gas,
                 value,
                 calldata=[]):
        self.pc      = 0
        
        self.stack   = Stack()
        self.memory  = Memory()
        self.storage = Storage()
        
        self.sender   = sender
        self.program  = program
        self.gas      = gas
        self.value    = value
        self.calldata = calldata
        
        self.stop_flag   = False
        self.revert_flag = False
        
        self.returndata = []
        self.logs       = []

In [2]:
# OpCode クラス
class OpCode(State):
    def __init__(self): 
        super.__init__()

    def stop(evm):
        evm.stop_flag = True
    
    def add(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(a+b)
        evm.pc += 1
        evm.gas_dec(3)
    
    def mul(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(a*b)
        evm.pc += 1
        evm.gas_dec(5)
    
    def sub(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(a-b)
        evm.pc += 1
        evm.gas_dec(3)

    def div(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(0 if b == 0 else a // b)
        evm.pc += 1
        evm.gas_dec(5)

    def sdiv(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        sign = pos_or_neg(a*b)
        evm.stack.push(0 if b == 0 else sign * (abs(a) // abs(b)))
        evm.pc += 1
        evm.gas_dec(5)
    
    def mod(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(0 if b == 0 else a % b)
        evm.pc += 1
        evm.gas_dec(5)
    
    def smod(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        sign = pos_or_neg(a*b)
        evm.stack.push(0 if b == 0 else abs(a) % abs(b) * sign)
        evm.pc += 1
        evm.gas_dec(5)
    
    def addmod(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        N = evm.stack.pop()
        evm.stack.push((a + b) % N)
        evm.pc += 1
        evm.gas_dec(8)

    def mulmod(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        N = evm.stack.pop()
        evm.stack.push((a + b) * N)
        evm.pc += 1
        evm.gas_dec(8)

    def size_in_bytes(number):
        import math
        if number == 0: return 1
        bits_needed = math.ceil(math.log2(abs(number) + 1))
        return math.ceil(bits_needed / 8)

    def exp(evm):
        a, exponent = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(a ** exponent)
        evm.pc += 1
        evm.gas_dec(10 + (50 * size_in_bytes(exponent)))

    def signextend(evm):
        b, x = evm.stack.pop(), evm.stack.pop()
        if b <= 31:
            testbit = b * 8 + 7
            sign_bit = 1 << testbit
            if x & sign_bit: result = x | (2**256 - sign_bit)
            else           : result = x & (sign_bit - 1)
        else: result = x
        
        evm.stack.push(result)
        evm.pc += 1
        evm.gas_dec(5)

    def lt(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(1 if a < b else 0)
        evm.pc += 1
        evm.gas_dec(3)

    def slt(evm): # signed less than
        a, b = evm.stack.pop(), evm.stack.pop()
        a = unsigned_to_signed(a)
        b = unsigned_to_signed(b)
        evm.stack.push(1 if a < b else 0)
        evm.pc += 1
        evm.gas_dec(3)

    def gt(evm): # greater than
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(1 if a > b else 0)
        evm.pc += 1
        evm.gas_dec(3)

    def sgt(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        a = unsigned_to_signed(a)
        b = unsigned_to_signed(b)
        evm.stack.push(1 if a > b else 0)
        evm.pc += 1
        evm.gas_dec(3)
    
    def eq(evm):
        a, b = cpu.stack.pop(), cpu.stack.pop()
        cpu.stack.push(1 if a == b else 0)
        cpu.pc += 1
        cpu.gas_dec(3)
    
    def iszero(evm):
        a = evm.stack.pop()
        evm.stack.push(1 if a == 0 else 0)
        evm.pc += 1
        evm.gas_dec(3)

    
    def _and(evm):
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(a & b)
        evm.pc += 1
        evm.gas_dec(3)
    
    def _or(evm): 
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(a | b)
        evm.pc += 1
        evm.gas_dec(3)

    def _xor(evm): 
        a, b = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(a ^ b)
        evm.pc += 1
        evm.gas_dec(3)
    
    def _not(evm): 
        a = evm.stack.pop()
        evm.stack.push(~a)
        evm.pc += 1
        evm.gas_dec(3)

    def byte(evm):
        i, x = evm.stack.pop(), evm.stack.pop()
        if i >= 32: result = 0
        else      : result = (x // pow(256, 31 - i)) % 256
        evm.pc += 1
        evm.gas_dec(3)
    
    def shl(evm): 
        shift, value = cpu.stack.pop(), cpu.stack.pop()
        evm.stack.push(value << shift)
        evm.pc += 1
        evm.gas_dec(3)
    
    def shr(evm): 
        shift, value = evm.stack.pop(), evm.stack.pop()
        evm.stack.push(value >> shift)
        evm.pc += 1
        evm.gas_dec(3)

    def sar(evm):
        shift, value = evm.stack.pop(), evm.stack.pop()
        if shift >= 256:
            result = 0 if value >= 0 else UINT_255_NEGATIVE_ONE
        else:
            result = (value >> shift) & UINT_256_MAX
            
        evm.stack.push(result)
        evm.pc += 1
        evm.gas_dec(3)
    
    def sha3(evm):
        offset, size = evm.stack.pop(), evm.stack.pop()
        value = evm.memory.access(offset, size)
        evm.stack.push(hash(str(value)))

        evm.pc += 1

        # calculate gas
        minimum_word_size = (size + 31) / 32
        dynamic_gas = 6 * minimum_word_size # TODO: + memory_expansion_cost
        evm.gas_dec(30 + dynamic_gas)

    def address(evm):
        evm.stack.push(evm.sender)
        evm.pc += 1
        evm.gas_dec(3)
    
    def balance(evm):
        address = evm.stack.pop()
        evm.stack.push(99999999999)

        evm.pc += 1
        evm.gas_dec(2600) # 100 if warm

    def origin(evm):
        evm.stack.push(evm.sender)
        evm.pc += 1
        evm.gas_dec(2)

    def caller(evm):
        evm.stack.push("0x414b60745072088d013721b4a28a0559b1A9d213")
        evm.pc += 1
        evm.gas_dec(2)

    def callvalue(evm):
        evm.stack.push(evm.value)
        evm.pc += 1
        evm.gas_dec(2)

    def calldataload(evm):
        i = evm.stack.pop()

        delta = 0
        if i+32 > len(evm.calldata):
            delta = i+32 - len(evm.calldata)

        # always has to be 32 bytes
        # if its not we append 0x00 bytes until it is
        calldata = evm.calldata[i:i+32-delta]
        calldata += 0x00*delta

        evm.stack.push(calldata)
        evm.pc += 1
        evm.gas_dec(3)

    def calldatasize(evm):
        evm.stack.push(len(evm.calldata))
        evm.pc += 1
        evm.gas_dec(2)
    
    def calldatacopy(evm):
        destOffset = evm.stack.pop()
        offset = evm.stack.pop()
        size = evm.stack.pop()

        calldata = evm.calldata[offset:offset+size]
        memory_expansion_cost = evm.memory.store(destOffset, calldata)

        static_gas = 3
        minimum_word_size = (size + 31) // 32
        dynamic_gas = 3 * minimum_word_size + memory_expansion_cost

        evm.gas_dec(static_gas + dynamic_gas)
        evm.pc += 1
    
    def codesize(evm):
        evm.stack.push(len(evm.program))
        evm.pc += 1
        evm.gas_dec(2)
    
    def codecopy(evm):
        destOffset = evm.stack.pop()
        offset     = evm.stack.pop()
        size       = evm.stack.pop()

        code = evm.program[offset:offset+size]
        memory_expansion_cost = evm.memory.store(destOffset, code)

        static_gas = 3
        minimum_word_size = (size + 31) / 32
        dynamic_gas = 3 * minimum_word_size + memory_expansion_cost

        evm.gas_dec(static_gas + dynamic_gas)
        evm.pc += 1

    def gasprice(evm):
        evm.stack.push(0x00)
        evm.pc += 1
        evm.gas_dec(2)
    
    def extcodesize(evm):
        address = evm.stack.pop()
        evm.stack.push(0x00)
        evm.gas_dec(2600) # 100 if warm
        evm.pc += 1

    def extcodecopy(evm):
        address    = evm.stack.pop()
        destOffset = evm.stack.pop()
        offset     = evm.stack.pop()
        size       = evm.stack.pop()

        extcode = [] # no external code
        memory_expansion_cost = evm.memory.store(destOffset, extcode)

        # refactor this in seperate method
        minimum_word_size = (size + 31) / 32
        dynamic_gas = 3 * minimum_word_size + memory_expansion_cost
        address_access_cost = 100 if warm else 2600

        evm.gas_dec(dynamic_gas + address_access_cost)
        evm.pc += 1

    def returndatasize(evm):
        evm.stack.push(0x00) # no return data
        evm.pc += 1
        evm.gas_dec(2)

    def returndatacopy(evm):
        destOffset = evm.stack.pop()
        offset     = evm.stack.pop()
        size       = evm.stack.pop()

        returndata            = evm.program[offset:offset+size]
        memory_expansion_cost = evm.memory.store(destOffset, returndata)

        minimum_word_size = (size + 31) / 32
        dynamic_gas = 3 * minimum_word_size + memory_expansion_cost

        evm.gas_dec(3 + dynamic_gas)
        evm.pc += 1

    def extcodehash(evm):
        address = evm.stack.pop()
        evm.stack.push(0x00) # no code

        evm.gas_dec(2600) # 100 if warm
        evm.pc += 1

    def blockhash(evm):
        blockNumber = evm.stack.pop()
        if blockNumber > 256: raise Exception("Only last 256 blocks can be accessed")
        evm.stack.push(0x1cbcfa1ffb1ca1ca8397d4f490194db5fc0543089b9dee43f76cf3f962a185e8)
        evm.pc += 1
        evm.gas_dec(20)

    def coinbase(evm):
        evm.stack.push(0x1cbcfa1ffb1ca1ca8397d4f490194db5fc0543089b9dee43f76cf3f962a185e8)
        evm.pc += 1
        evm.gas_dec(2)

    def _pop(evm):
        evm.pc += 2
        evm.gas_dec(2)
        evm.stack.pop(0)

    def mload(evm): 
        offset = evm.stack.pop()
        value = evm.memory.load(offset)
        evm.stack.push(value)
        evm.pc += 1
    
    def mstore(evm): 
        # TODO: should be right aligned
        offset, value = evm.stack.pop(), cpu.stack.pop()
        evm.memory.store(offset, value)
        evm.pc += 1

    def mstore8(evm): 
        offset, value = evm.stack.pop(), evm.stack.pop()
        evm.memory.store(offset, value)
        evm.pc += 1

    def sload(evm): 
        key = evm.stack.pop().value
        warm, value = evm.storage.load(key)
        evm.stack.push(value)

        evm.gas_dec(2100) # 100 if warm
        evm.pc += 1

    def sstore(evm): 
        key, value = evm.stack.pop(), evm.stack.pop()
        warm, old_value = evm.storage.store(key, value)

        base_dynamic_gas = 0

        if value != old_value:
            if old_value == 0:
                base_dynamic_gas = 20000
            else:
                base_dynamic_gas = 2900

        access_cost = 100 if warm else 2100
        evm.gas_dec(base_dynamic_gas + access_cost)

        evm.pc += 1

    def tload(evm): 
        key = evm.stack.pop().value
        warm, value = evm.storage.load(key)
        evm.stack.push(value)

        evm.gas_dec(100)
        evm.pc += 1

    def tstore(evm): 
        key, value = evm.stack.pop(), evm.stack.pop()
        evm.storage.store(key, value)
        evm.gas_dec(100)
        evm.pc += 1
    
    def jump(evm):
        counter = evm.stack.pop()

        # make sure that we jump to an JUMPDEST opcode
        if not evm.program[counter] == JUMPDEST:
            raise Exception("Can only jump to JUMPDEST")

        evm.pc = counter
        evm.gas_dec(8)

    def jumpi(evm):
        counter, b = evm.stack.pop(), evm.stack.pop()

        if b != 0: evm.pc = counter
        else     : evm.pc += 1
        
        evm.gas_dec(10)

    def pc(evm):
        evm.stack.push(cpu.pc)
        evm.pc += 1
        evm.gas_dec(2)

    def jumpdest(evm):
        evm.pc += 1
        evm.gas_dec(1)

    def _push(evm, n):
        evm.pc += 1
        evm.gas_dec(3)
        
        value = []
        for _ in range(n):
            value.append(evm.peek())
            evm.pc += 1
        evm.stack.push(int(''.join(map(str, value))))

    def _dup(evm, n):
        # make sure stack is big enough!
        value = evm.stack[n]
        evm.stack.push(value)

        evm.pc += 1
        evm.gas_dec(3)
    
    def _swap(evm, n):
        value1, value2 = evm.stack.get(0), evm.stack.get(n+1)
        evm.stack.set(0, value2)
        evm.stack.set(n+1, value1)

        evm.pc += 1
        evm.gas_dec(3)

    def revert(evm):
        offset, size = evm.stack.pop(), evm.stack.pop()
        evm.returndata = evm.memory.access(offset, size)

        evm.stop_flag = True
        evm.revert_flag = True
        evm.pc += 1
        evm.gas_dec(0)

class Log:
    def __init__(self,
                 data,
                 topic1=None,
                 topic2=None,
                 topic3=None,
                 topic4=None):
        
        self.data = data
        self.topic1 = topic1
        self.topic2 = topic2
        self.topic3 = topic3
        self.topic4 = topic4

    def __str__(self): return f"Log: {self.data}"

    def calc_gas(topic_count, size, memory_expansion_cost=0):
        # 375 := static_gas
        return 375 * topic_count + 8 * size + memory_expansion_cost

    def log0(evm):
        offset, size = evm.stack.pop(), evm.stack.pop()

        data = evm.memory.access(offset, size)
        log = Log(data)
        evm.append_log(log)

        evm.pc += 1
        evm.gas_dec(calc_gas(0, size)) # TODO: memory expansion cost

    def log1(evm):
        offset, size = evm.stack.pop(), evm.stack.pop()
        topic = evm.stack.pop().value

        data = evm.memory.access(offset, size)
        log = Log(data, topic)
        evm.append_log(log)

        evm.pc += 1
        evm.gas_dec(calc_gas(1, size)) # TODO: memory expansion cost

    def log2(evm):
        offset, size = evm.stack.pop(), evm.stack.pop()
        topic1, topic2 = evm.stack.pop(), evm.stack.pop()

        data = evm.memory.access(offset, size)
        log = Log(data, topic1, topic2)
        evm.append_log(log)

        evm.pc += 1
        evm.gas_dec(calc_gas(2, size)) # TODO: memory expansion cost

    def log3(evm):
        offset, size = evm.stack.pop(), evm.stack.pop()
        topic1 = evm.stack.pop()
        topic2 = evm.stack.pop()
        topic3 = evm.stack.pop()

        data = evm.memory.access(offset, size)
        log = Log(data, topic1, topic2, topic3)
        evm.append_log(log)

        evm.pc += 1
        evm.gas_dec(calc_gas(3, size)) # TODO: memory expansion cost

    def log4(cpu):
        offset, size = evm.stack.pop(), evm.stack.pop()
        topic1 = evm.stack.pop()
        topic2 = evm.stack.pop()
        topic3 = evm.stack.pop()
        topic4 = evm.stack.pop()

        data = evm.memory.access(offset, size)
        log = Log(data, topic1, topic2, topic3, topic4)
        evm.append_log(log)

        evm.pc += 1
        evm.gas_dec(calc_gas(4, size)) # TODO: memory expansion cost

In [ ]:
class EVM:
    def __init__(self,
                 program,
                 gas,
                 value,
                 calldata=[]):
        self.pc      = 0
        self.stack   = Stack()
        self.memory  = Memory()
        self.storage = Storage()
        
        self.program  = program
        self.gas      = gas
        self.value    = value
        self.calldata = calldata
        
        self.stop_flag   = False
        self.revert_flag = False
        
        self.returndata = []
        self.logs       = []
        
    def peek(self): return self.program[self.pc]
    
    def gas_dec(self, amount):
        if self.gas - amount < 0: 
            raise Exception("out of gas")
        self.gas -= amount

    
    def should_execute_next_opcode(self):
        if self.pc > len(self.program)-1: return False
        if self.stop_flag               : return False
        if self.revert_flag             : return False
        
        return True
    
    def run(self):
        while self.should_execute_next_opcode():
            op = self.program[self.pc]
            
            if op == STOP: stop(self)
                
            if op == ADD: add(self)

                
            if op == PUSH1: _push(self, 1)
        
    def reset(self):
        self.pc      = 0
        self.stack   = Stack()
        self.memory  = Memory()
        self.storage = Storage()